In [29]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf

from matplotlib import pyplot as plt
from sklearn import preprocessing
from IPython.display import display

In [30]:
pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.3f}".format
path = './saved_models/Reg/h5/'

In [31]:
#importar dataset
training_df = pd.read_csv('train_validation.csv')
test_df = pd.read_csv('test.csv')

In [32]:
#aleatorize el dataset
shuffled_training_df = training_df.reindex(np.random.permutation(training_df.index))
shuffled_test_df = test_df.reindex(np.random.permutation(test_df.index))
shuffled_test_df

,wconfid,pctid,x,y,z
91957,2,85,0.641,1.848,1.176
41564,1,85,-2.336,0.230,0.484
27201,1,65,1.223,0.434,0.305
78255,2,65,1.195,0.043,-0.012
6794,1,30,0.992,0.020,-0.113
...,...,...,...,...,...
88428,2,80,0.766,0.121,0.637
95812,2,90,1.094,1.094,-0.512
13488,1,40,0.961,0.035,-0.109
99609,2,100,1.066,1.012,-0.973


In [33]:
scaler = preprocessing.MinMaxScaler(feature_range=(0,1))

norm_training = pd.DataFrame(scaler.fit_transform(shuffled_training_df))
norm_training.columns=['wconfid','pctid','x','y','z']

norm_test = pd.DataFrame(scaler.fit_transform(shuffled_test_df))
norm_test.columns=['wconfid','pctid','x','y','z']

inv = scaler.fit(shuffled_test_df)
display(norm_test.describe())

,wconfid,pctid,x,y,z
count,153000.000,153000.000,153000.000,153000.000,153000.000
mean,0.500,0.500,0.562,0.500,0.481
std,0.408,0.306,0.048,0.046,0.043
min,0.000,0.000,0.000,0.000,0.000
25%,0.000,0.250,0.559,0.495,0.476
50%,0.500,0.500,0.562,0.501,0.480
75%,1.000,0.750,0.565,0.507,0.485
max,1.000,1.000,1.000,1.000,1.000


In [34]:
train_features = norm_training[['pctid','x','y','z']]
train_labels = norm_training[['wconfid']] 

test_features = norm_test[['pctid','x','y','z']]
test_labels = norm_test[['wconfid']] 
display(test_features)
display(test_labels)

,pctid,x,y,z
0,0.812,0.540,0.616,0.589
1,0.812,0.354,0.515,0.531
2,0.562,0.577,0.527,0.516
3,0.562,0.575,0.503,0.490
4,0.125,0.562,0.501,0.481
...,...,...,...,...
152995,0.750,0.548,0.508,0.544
152996,0.875,0.569,0.569,0.448
152997,0.250,0.560,0.502,0.482
152998,1.000,0.567,0.563,0.409


,wconfid
0,0.500
1,0.000
2,0.000
3,0.500
4,0.000
...,...
152995,0.500
152996,0.500
152997,0.000
152998,0.500


In [35]:
num_classes = 3
def my_model(my_learning_rate):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(3, input_shape=(4,), activation='sigmoid'),
        tf.keras.layers.Dense(num_classes)
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=my_learning_rate),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model

model = my_model(0.01)
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 3)                 15        
                                                                 
 dense_11 (Dense)            (None, 3)                 12        
                                                                 
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


In [36]:
epochs = 10
model = my_model(0.01)
history = model.fit(
    x=train_features,
    y= train_labels,
    epochs = epochs
    
)

Epoch 1/10
3825/3825 [==============================] - 3s 665us/step - loss: 0.6796 - accuracy: 0.2337
Epoch 2/10
3825/3825 [==============================] - 2s 622us/step - loss: 0.6701 - accuracy: 0.2567
Epoch 3/10
3825/3825 [==============================] - 3s 686us/step - loss: 0.6698 - accuracy: 0.2562
Epoch 4/10
3825/3825 [==============================] - 3s 680us/step - loss: 0.6695 - accuracy: 0.2563
Epoch 5/10
3825/3825 [==============================] - 3s 678us/step - loss: 0.6686 - accuracy: 0.2585
Epoch 6/10
3825/3825 [==============================] - 3s 674us/step - loss: 0.6670 - accuracy: 0.2611
Epoch 7/10
3825/3825 [==============================] - 3s 721us/step - loss: 0.6640 - accuracy: 0.2594
Epoch 8/10
3825/3825 [==============================] - 3s 867us/step - loss: 0.6612 - accuracy: 0.2599
Epoch 9/10
3825/3825 [==============================] - 3s 697us/step - loss: 0.6549 - accuracy: 0.2610
Epoch 10/10
3825/3825 [==============================] - 3s 665u

In [37]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


KeyError: 'val_accuracy'